In [187]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [188]:
load_dotenv()
MODEL = "gpt-4.1-mini"
client = OpenAI()

In [189]:
system_message ="""
You are English Vocabulary Trainer.You will give and explain the meaning of the word.
You will start conversation with any word u want to teach user, Initially user mostly will not ask for word, so it will be better if u start it.
You will also give a sentence using the word.
You will also give the antonym and synonym of the word.
Search the word in the database ,if found dont update the database and give the result.If absent create a new entry.You have tools to do so.
You will give it in following format:
Word: <word>
Meaning: <meaning>
Sentence: <sentence>
Antonym: <antonym>
Synonym: <synonym>

"""

In [190]:
import sqlite3

In [191]:
DB ="vocabulary.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS vocabulary (word TEXT PRIMARY KEY, meaning TEXT, sentence TEXT, antonym TEXT, synonym TEXT)')




In [192]:
def get_vocabulary(word):
    results =[]
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT word,sentence FROM vocabulary WHERE word=?', (word.lower(),)) 
        result = cursor.fetchone()
        if result:
            results.append(result[1])
            return f"the result is word {result[0]} and sentence {result[1]} after fetching from the database",results
        else:
            return f"the word {word} is not present in the database",results
def set_vocabulary(word, meaning, sentence, antonym, synonym):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO vocabulary (word, meaning, sentence, antonym, synonym) VALUES (?, ?, ?, ?, ?)', (word, meaning, sentence, antonym, synonym))
        conn.commit()



In [193]:
get_vocabulary_from_db_function = {
    "name": "get_vocabulary",
    "description": "Get the vocabulary from the database",
    "parameters": {
        "type": "object",
        "properties": {
            "word": {
                "type": "string",
                "description": "we will use this function to check whether the word is present in the database or not"
            }
        },
        "required": ["word"]
    }
}

set_vocabulary_in_db_function = {
    "name": "set_vocabulary",
    "description": "Set the vocabulary in the database",
    "parameters": {
        "type": "object",
        "properties": {
            "word": {
                "type": "string",
                "description": "The word to set the vocabulary for"
            },
            "meaning": {
                "type": "string",
                "description": "The meaning of the word"
            },
            "sentence": {
                "type": "string",
                "description": "The sentence using the word"
            },
            "antonym": {
                "type": "string",
                "description": "The antonym of the word"
            },
            "synonym": {
                "type": "string",
                "description": "The synonym of the word"
            }
        },
        "required": ["word", "meaning", "sentence", "antonym", "synonym"]
    }
}






In [194]:
tools = [{"type": "function", "function":get_vocabulary_from_db_function},{"type": "function", "function": set_vocabulary_in_db_function}]

In [195]:
def handle_tool_calls(message):
    respones = []
    words = None
    sentences = None
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_vocabulary":
            print("get_vocabulary tool call")
            arguments = json.loads(tool_call.function.arguments)
            word = arguments.get('word')
            result,answer = get_vocabulary(word)
            words=word
            if answer:
                sentences=answer
            print(word)    
            print(sentences)
            respones.append({
                "role": "tool",
                "content": result,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "set_vocabulary":
            print("set_vocabulary tool call")
            arguments = json.loads(tool_call.function.arguments)
            word = arguments.get('word')
            meaning = arguments.get('meaning')
            sentence = arguments.get('sentence')
            antonym = arguments.get('antonym')
            synonym = arguments.get('synonym')
            words=word
            if answer:
                sentences=answer
            set_vocabulary(word, meaning, sentence, antonym, synonym)
            respones.append({
                "role": "tool",
                "content":"Vocabulary & details for {word} is set successfully",
                "tool_call_id": tool_call.id
            })
    print(sentences)
    return respones, words, sentences




In [196]:
def chat(history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history
    response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    wordes = None
    sentences = None
    image = None
    while response.choices[0].finish_reason=="tool_calls":
        print("tool call")
        message = response.choices[0].message
        responses, words ,sentence = handle_tool_calls(message)
        if sentence:
            sentences=sentence
        if words:
            wordes=words
        messages.append(message)
        messages.extend(responses)
        response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    voice = talker(reply)
    print("words and sentences",wordes,sentences)
    if wordes and sentences:
        print("words and sentences",wordes,sentences)
        image = artist(wordes[0]+"example sentence:"+sentences[0])
    
    return history, image, voice

In [197]:

import base64
from io import BytesIO
from PIL import Image

In [198]:
def artist(word):
    image_response = client.images.generate(
            model="dall-e-3",
            prompt=f"An image word & sentence {word},we are making user to learn the word by showing the image",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [199]:
def talker(message):
    response = client.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="onyx",    # Also, try replacing onyx with alloy or coral
      input=message
    )
    return response.content

In [ ]:

def put_message_in_chatbot(message, history):
        return "", history + [{"role":"user", "content":message}]

# UI definition

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")

# Hooking up events to callbacks

    message.submit(put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output,audio_output]
    )

ui.launch(inbrowser=False)

/var/folders/t0/v7t1g6px5_qdwtfr81fbsrf40000gn/T/ipykernel_9076/3504710964.py:8: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(height=500, type="messages")


* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.


tool call
get_vocabulary tool call
serendipity
['Finding my old friend in a foreign country was pure serendipity.']
['Finding my old friend in a foreign country was pure serendipity.']
words and sentences serendipity ['Finding my old friend in a foreign country was pure serendipity.']
words and sentences serendipity ['Finding my old friend in a foreign country was pure serendipity.']
